# load HieroglyphDB from .json

In [1]:
from hieroglyph_class import *
import pickle
import json
from enum import Enum
from dataclasses import is_dataclass, asdict

def load_hieroglyphDB_from_json(filename: str) -> HieroglyphDB:
  def dict_to_reading(d: dict) -> Reading:
    return Reading(
        onyomi=d["onyomi"],
        kunyomi=d["kunyomi"],
        vocab=d["vocab"],
        main_reading=d["main_reading"]
      )

  def dict_to_mnemonics(d: dict) -> Mnemonics:
    return Mnemonics(
      meaning=d["meaning"],
      reading=d["reading"]
    )

  def dict_to_resource_paths(d: dict) -> ResourcePaths:
    return ResourcePaths(
      pic=d["pic"],
      sound=d["sound"],
      wanikani_link=d["wanikani_link"]
    )

  def dict_to_hieroglyph(d: dict) -> Hieroglyph:
    hieroglyph_type = HieroglyphType(d["hieroglyph_type"])
    reading = dict_to_reading(d["readings"])
    mnemonics = dict_to_mnemonics(d["mnemonics"])
    resource_paths = dict_to_resource_paths(d["resource_paths"])
    sentences = [tuple(s) for s in d["sentences"]]

    return Hieroglyph(
      symbol=d["symbol"],
      level=d["level"],
      hieroglyph_type=hieroglyph_type,
      meanings=d["meanings"],
      readings=reading,
      mnemonics=mnemonics,
      sentences=sentences,
      resource_paths=resource_paths
    )

  def dict_to_hieroglyphDB(d: dict) -> HieroglyphDB:
    hieroglyphs = [dict_to_hieroglyph(h) for h in d["hieroglyphs"]]
    return HieroglyphDB(hieroglyphs=hieroglyphs)

  # Load the JSON file
  with open(filename, "r", encoding="utf-8") as f:
    data = json.load(f)

  # Reconstruct HieroglyphDB from the loaded dictionary
  return dict_to_hieroglyphDB(data)


db = load_hieroglyphDB_from_json("../HieroglyphDB.json")
# with open('HieroglyphDB.pkl', 'wb') as f:
#   pickle.dump(db, f)

# dump HieroglyphDB to .json

In [41]:
from hieroglyph_class import *
import pickle
import json
from enum import Enum
from dataclasses import is_dataclass, asdict

def enum_to_value(obj):
  # Converts enum instances to their value
  return obj.value if isinstance(obj, Enum) else obj

def dataclass_to_dict(obj):
  # Recursively converts dataclasses (and lists, tuples) to dict for JSON serialization
  if is_dataclass(obj):
    result = {}
    for field_name in obj.__dataclass_fields__:
      field_value = getattr(obj, field_name)
      result[field_name] = dataclass_to_dict(field_value)
    return result
  elif isinstance(obj, list):
    return [dataclass_to_dict(item) for item in obj]
  elif isinstance(obj, tuple):
    return tuple(dataclass_to_dict(item) for item in obj)
  elif isinstance(obj, Enum):
    return enum_to_value(obj)
  else:
    return obj

# # Load the pickled HieroglyphDB
# with open('HieroglyphDB.pkl', 'rb') as f:
#   db: HieroglyphDB = pickle.load(f)

# Convert to JSON-serializable dictionary
db_dict = dataclass_to_dict(db)

# Write out to JSON
with open('../HieroglyphDB.json', 'w', encoding='utf-8') as f:
  json.dump(db_dict, f, ensure_ascii=False, indent=2)

print("HieroglyphDB successfully converted to HieroglyphDB.json")

HieroglyphDB successfully converted to HieroglyphDB.json


# separate HieroglyphDB on chunks

In [93]:
import pickle
from hieroglyph_class import *

with open('HieroglyphDB.pkl', 'rb') as f:
  db = pickle.load(f)

db_lvls = {i:[] for i in range(0,60,5)}
for i in db:
  db_lvls[(i.level-1)//5 * 5].append(i)

for i in db_lvls:
  db_dict = dataclass_to_dict(HieroglyphDB(db_lvls[i]))
  with open(f'../jsons/HieroglyphDB_{i}.json', 'w', encoding='utf-8') as f:
    json.dump(db_dict, f, ensure_ascii=False, indent=2)
